In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
dataset, info = tfds.load('cats_vs_dogs', with_info=True, as_supervised=True)

In [ ]:
class_names = info.features['label'].names
class_names

In [ ]:
for i , example in enumerate(dataset['train']):
  image, label = example
  save_dir ='cats_vs_dogs/train/{}'.format(class_names[label])
  os.makedirs(save_dir, exist_ok=True)

  filename=save_dir + '/' + "{}_{}.jpg".format(class_names[label], i)
  tf.keras.preprocessing.image.save_img(filename, image.numpy())

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential

In [ ]:
datagen = ImageDataGenerator(rescale = 1/255 , validation_split=0.2, rotation_range=10,
                             width_shift_range=0.1, height_shift_range=0.1, shear_range=0.1, zoom_range=0.10,
                             horizontal_flip = True)



train_generator = datagen.flow_from_directory(
        '/content/cats_vs_dogs/train',
        target_size=(150, 150),
        batch_size=216,
        class_mode='binary',
        subset='training')

validation_generator = datagen.flow_from_directory(
        '/content/cats_vs_dogs/train',
        target_size=(150, 150),
        batch_size=216,
        class_mode='binary',
        subset='validation')

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    BatchNormalization(),
    Dropout(0.2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    BatchNormalization(),
    Dropout(0.2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    BatchNormalization(),
    Dropout(0.2),

    Flatten(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(
      train_generator,
      steps_per_epoch=len(train_generator),
      epochs=10,
      validation_data=validation_generator,
      validation_steps=len(validation_generator)
)

In [ ]:
model.summary()

In [ ]:
history.history

plt.plot(history.history['accuracy'], label='Traning')
plt.plot(history.history['val_accuracy'], label='Validation')
plt.legend(['Traning','Validation'])

In [ ]:
model.save('cats_vs_dogs.h5')

In [ ]:
model_load = tf.keras.models.load_model('cats_vs_dogs.h5')

In [ ]:
import requests
from PIL import Image

import numpy as np
import requests
from PIL import Image
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image

def predict_and_show_image(img_url, model, class_indices, target_size=(150, 150), threshold=0.5):
    # Load and resize image
    img = Image.open(requests.get(img_url, stream=True).raw).resize(target_size)

    # Preprocess image
    image_array = image.img_to_array(img)
    img_array = np.expand_dims(image_array, axis=0) / 255.0

    # Make prediction
    prediction = model.predict(img_array)

    # For binary classification
    predicted_class_index = int(prediction[0][0] > threshold)

    # Reverse class indices to get label names
    classes = {v: k for k, v in class_indices.items()}
    predicted_class = classes[predicted_class_index]

    # Display the image and prediction
    plt.imshow(image_array.astype("uint8"))
    plt.title(f"Predicted Class: {predicted_class}")
    plt.axis('off')
    plt.show()

    return predicted_class

In [ ]:
img_url1 = 'https://images.pexels.com/photos/1490908/pexels-photo-1490908.jpeg?cs=srgb&dl=pexels-svetozar-milashevich-99573-1490908.jpg&fm=jpg'
predict_and_show_image(img_url1, model, train_generator.class_indices)

In [ ]:
img_url2 = 'https://plus.unsplash.com/premium_photo-1667030474693-6d0632f97029?w=600&auto=format&fit=crop&q=60&ixlib=rb-4.1.0&ixid=M3wxMjA3fDB8MHxzZWFyY2h8MXx8Y2F0fGVufDB8fDB8fHww'
predict_and_show_image(img_url2, model, train_generator.class_indices)